In [1]:
import tensorflow as tf

In [ ]:
import os
#op.path.join is used to give the path of the image irrespective of the os
#folder structure is given
#os.listdir is used to get all of the files of a particular dir
from matplotlib import pyplot as plt

In [ ]:
!pip list

In [ ]:
gpus=tf.config.experimental.list_physical_devices('CPU')

In [ ]:
gpus

In [ ]:
len(gpus)

In [ ]:
#limit gpu from utilizing all of the memory-when u take large data set
#all gpus are taken-no of gpus -devices that are available can be collected and displayed
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)
    #tells tensorflow to limit the memory and not use up all

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir='data'

In [ ]:
os.listdir(os.path.join(data_dir,'oily'))

In [ ]:
image_exts=['jpeg','jpg','bmp','png']

In [ ]:
image_exts

In [ ]:
for image_class in os.listdir(data_dir):
    print(image_class)

In [ ]:
from pathlib import Path
import imghdr

image_extensions = [".jpg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir,image_class)):
        print(image)

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir,image_class)):
        image_path=os.path.join(data_dir,image_class,image)
        try:
            img=cv2.imread(image_path)
            tip=imgdr.what(image_path)
            if tip not in image_exts:
                print("Image not in ext list{}".format(image_path))
                os.remove(image_path)
        except Exception as e:
            print("Issue with image {}".format(image_path))

In [ ]:
cv2.imread(os.path.join('data','oily','1.jpg'))

In [ ]:
img.shape

In [ ]:
#LOAD OUT DATA SET
#tensor flow has an api that helps u build data pipeline-scales large data set
#makes stuff cleanes
tf.data.Dataset??
#lot of capability

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [ ]:
plt.show()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data=tf.keras.utils.image_dataset_from_directory??
#?? is for documentation

In [ ]:
#data pipeline builtin
#labels and classes need not be built
#resizing included
data=tf.keras.utils.image_dataset_from_directory('data')

In [ ]:
data

In [ ]:
#data set is a generator
#convert into numpy iterator
data_iterator=data.as_numpy_iterator()
#the above line allows us to access data from the pipeline on the go

In [ ]:
data_iterator
#loop through and pull data-in massive amts of data

In [ ]:
#acces pipeline
batch=data_iterator.next()
#now we get the data in the req part

In [ ]:
batch

In [ ]:
len(batch)
#2 parts created to the data set
#part 1-images
#part2-labels

In [ ]:
batch[0]

In [ ]:
batch[0].shape
#automatically reshapes the images to make it more compatable
#batch size and all the others can be changed

In [ ]:
batch[1]
#0 represents different

In [ ]:
fig,ax=plt.subplots(ncols=4,figsize=(20,20))
for idx,img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])
#now we get 0-dry skin and 1-oily skin
#batch[0] cantains images
#batch[1] contains labels

In [ ]:
#the images are in the values 0 to 255-rgb format
batch[0].max()

In [ ]:
batch[0].min()

In [ ]:
#to make our classifier better we need to make the rgb values as small as possible-faster
#therefore divi by 255
scaled=batch[0]/255
#leaves values between 0 and 1

In [ ]:
scaled.max()

In [ ]:
#PREPROCESSING OF DATA
#preprocess part1 - scaling
#preprocess part2 - training

In [ ]:
#scaling part
#pipeling map fn apply transformation can be applied as our data is loaded to our pipeline
#speeds up the process of accessing our data from the disc
data = data.map(lambda x,y:(x/255,y))
#data.map-trandsformation in pipeline
#x-images and y-target variables

In [ ]:
#tensorflow data api helps in a ton functions across the data set
#inside of our data pipeline
scaled_iterator=data.as_numpy_iterator()

In [ ]:
scaled_iterator.next()

In [ ]:
scaled_iterator.next()[0].max()

In [ ]:
#splitting our data
len(data)

In [ ]:
#we'll make train 70% of that data so the split ratio we'll make it 70:30
#val is forvalidation

In [ ]:
train_size=int(len(data)*.7)
val_size=int(len(data)*.2)+1 #evaluation while training
test_size=int(len(data)*.1)+1

In [ ]:
val_size+train_size+test_size#allocation of the data to various partition

In [ ]:
#take and skip data from tensorflow
#take defines how much data we take for each part
#skip-avoids already taken part
train=data.take(train_size)
val=data.skip(train_size).take(val_size)
test=data.skip(train_size+val_size).take(test_size)

In [ ]:
len(test)
#shuffle the data by avoiding the already allocated batches

In [ ]:
#DEEP LEARNING MODEL
#ai part
#Part 1-deep learning model-nn
#Part 2-training the model 
#Part 3-consists of the performance plots

In [ ]:
#part 1

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense

In [ ]:
#model building api from tensor flow we use sequential-quick and easy
#this is very straightfroward that like with one data input and one data output
#the alternate option being the functional api thats very powerful that takes in multiple inputs and gives multiple outputs
#conv2d is the cnn spatial conveolution over pics
#maxPooling-condensing layer
#Dense-is fully connected layer
#flatten the convolution that goes to kernels that give output
#dropout not that useful for regularization

In [ ]:
#architecture,add method is another
model=Sequential()

In [ ]:
#model is adding conv layer and maxPooling layer
#first layer must be input layer
#conv2d has 16 filters here it scans the images and extracts/condenses the info from the pictures inside it .
#stride-1 here it goes pixel by pixel
#3*3 isthe size of the filters
#how we make architectural changes-model parameters changes decide the performance of the models
#activation -relu output is passed to fn that makes output below 0 makes the values and positive values are preserved
#sigmoid activation the outpus are modified by passing it to a graphical curve
#changing the function makes the output 
#maxpooling takes a set of number 2*2 region it goes through reduces info by half

model.add(Conv2D(16,(3,3),1,activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D())

#next layers added

model.add(Conv2D(32,(3,3),1,activation="relu"))
model.add(MaxPooling2D())

model.add(Conv2D(16,(3,3),1,activation="relu"))
model.add(MaxPooling2D())

#when we applu the conv layers the filters are gonna be the last channels-we dont want channel value we need 1 value as output

model.add(Flatten())

#fully connected layers
#final layer is 256 parts
model.add(Dense(256,activation="relu"))

#takes in a range of values and makes the vaue between 0 and 1 values.This is basically the output that we are expected to give
model.add(Dense(1,activation="sigmoid"))
#0-dry and 1-oily

In [ ]:
#next most important part
#optimizer-adam(there are many to choose from)
#loss is for a binary classifier-accuracy is taken into consideration to check the model's performance


In [ ]:
model.compile('adam',loss=tf.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [ ]:
#summary fow model transforms our data
model.summary()

In [ ]:
#part 2-training

In [ ]:
#variable that points to logs folder
logdir="logs"

In [ ]:
#callback calls the pointed out place-logbacks to the models training
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
#fit and predict are important parts of thetraining models-
#fit from training data-epochs no of parts on training parts
#we could also see how its training

In [ ]:
hist=model.fit(train,epochs=20,validation_data=val,callbacks=[tensorboard_callback])

In [ ]:
#gpu makes taining part fast-loss decreases and accuracy increases
#we could also make the the model without using gpu's
#later the hist value could be for sure used find the other logs
hist

In [ ]:
hist.history
#loss and accuracy info can be optained

In [ ]:
#part 3 -plot performance

In [ ]:
#-training loss and val loss plotted
fig=plt.figure()
plt.plot(hist.history['loss'],color='teal',label='loss')
plt.plot(hist.history['val_loss'],color='orange',label='val_loss')
#fig.subtitle('Loss',fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
#val score needs to decrease-regularization needs to be done

In [ ]:
#accuracy visualised
fig=plt.figure()
plt.plot(hist.history['accuracy'],color='teal',label='accuracy')
plt.plot(hist.history['val_accuracy'],color='orange',label='val_accuracy')
#fig.subtitle('Loss',fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
#now ths is our trained model

In [ ]:
#PART 2:EVALUATION OF PERFORMANCE
#key metrics are used

In [ ]:
from tensorflow.keras.metrics import Precision,Recall,BinaryAccuracy

In [ ]:
pre=Precision()
re=Recall()
acc=BinaryAccuracy()

In [ ]:
#test them
len(test)

In [ ]:
for batch in test.as_numpy_iterator():
    x,y=batch
    yhat=model.predict(x)
    pre.update_state(y,yhat)
    re.update_state(y,yhat)
    acc.update_state(y,yhat)

In [ ]:
print(f"Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}")

In [ ]:
#the values present th higher the better

In [ ]:
#Part2 Testing
import cv2
#open cv library

In [ ]:
pip install opencv-python

In [ ]:
img=cv2.imread('dry.sample.jpg')
#color correction
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
#resize before pushing it nn
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
resize=tf.image.resize(rgb_img,(256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
resize

In [ ]:
resize.shape

In [ ]:
np.expand_dims(resize,0).shape

In [ ]:
yhat=model.predict(np.expand_dims(resize/255,0))

In [ ]:
yhat

In [ ]:
#encapsulation by putting it into the extradimen is stored

In [ ]:
if yhat>0.5:
    print(f'Predicted class is Oily Skin')
else:
    print(f'Predicted class is Dry Skin')

In [ ]:
#SAVE THE MODEL THE FINAL!! PART
#to make the model to be available to other developers

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','skinclassification.hdf5'))
#serialization-load model can be used to do that

In [ ]:
#reload the model back up
new_model=load_model(os.path.join('models','skinclassification.hdf5'))

In [ ]:
new_model

In [ ]:
yhatnew=new_model.predict(np.expand_dims(resize/255,0))

In [ ]:
if yhatnew>0.5:
    print("oily skin")
else:
    print("dry skin")

In [ ]:
!pip install gradio

In [ ]:
import gradio

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [ ]:
pip install gradio

In [ ]:
import gradio as gr

# def greet(name):
#     return "Hello " + name + "!"

# demo = gr.Interface(fn=greet, inputs="text", outputs="text")
    
# if __name__ == "__main__":
#     demo.launch(show_api=False) 

In [ ]:
import gradio as gr

In [ ]:
def predict_image(img):
    img_3d=img.reshape(1, 256, 256, 3)
    prediction=model.predict(img_3d)[0]
    return {class_names[i]:float(prediction[i]) for i in range(2)}

In [ ]:
demo = gr.Interface(sepia, gr.Image(), "image")


In [ ]:
import numpy as np
import gradio as gr

def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189], 
        [0.349, 0.686, 0.168], 
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

demo = gr.Interface(sepia, gr.Image(), "image")
demo.launch()

In [ ]:
new_model=load_model(os.path.join('models','skinclassification.h5'))

In [ ]:
new_model

In [ ]:
if yhatnew>0.5:
    print("oily skin")
else:
    print("dry skin")